# NBA_API Contents Analysis 
I'm going to go through every package available in the NBA_API app to see what I can use.  

In [1]:
# imports 
import pandas as pd
import numpy as np

## Static 
This is where we get things like player id, team id, game id... etc.. Basically, stuff we will use later. 

### Teams

In [2]:
from nba_api.stats.static import teams
team_list = teams.get_teams()
team_df = pd.DataFrame(team_list)
team_df.shape

(30, 7)

In [3]:
team_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


### Players

In [75]:
from nba_api.stats.static import players
player_list = players.get_players()
player_df = pd.DataFrame(player_list)
print(player_df.shape)

(4815, 5)


In [76]:
player_df.to_csv('players.csv',index=False)

In [5]:
player_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


### Determining Test Cases  
We need some test teams and test players to proceed in the next step. I want two teams, two active players, and two inactive players.   

In [6]:
team_df[(team_df['full_name']=='Dallas Mavericks')|(team_df['full_name']=='Charlotte Hornets')]

,id,full_name,abbreviation,nickname,city,state,year_founded
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
29,1610612766,Charlotte Hornets,CHA,Hornets,Charlotte,North Carolina,1988


In [7]:
tid1 = '1610612742'
tid2 = '1610612766'

In [8]:
player_df[(player_df['full_name']=='Dirk Nowitzki')|(player_df['full_name']=='Michael Jordan')|(player_df['full_name']=='LeBron James')|(player_df['full_name']=='Luka Doncic')]

,id,full_name,first_name,last_name,is_active
1085,1629029,Luka Doncic,Luka,Doncic,True
2075,2544,LeBron James,LeBron,James,True
2252,893,Michael Jordan,Michael,Jordan,False
3172,1717,Dirk Nowitzki,Dirk,Nowitzki,False


In [9]:
pid1 = '1629029'
pid2 = '2544'
pid3 = '893'
pid4 = '1717'

In [10]:
# I need some game_ids... 
gid1 = '0021800283'

## Endpoints  
These are the actual stats 

### alltimeleadersgrids.md

In [19]:
from nba_api.stats.endpoints import alltimeleadersgrids

In [20]:
atlg = alltimeleadersgrids.AllTimeLeadersGrids(league_id='00',topx=10).get_data_frames()
atlg[0]
# league_id = '00' is for the NBA  
# I can get however many I want 

,PLAYER_ID,PLAYER_NAME,GP,GP_RANK,IS_ACTIVE_FLAG
0,305,Robert Parish,1611,1,N
1,76003,Kareem Abdul-Jabbar,1560,2,N
2,1713,Vince Carter,1541,3,N
3,1717,Dirk Nowitzki,1522,4,N
4,304,John Stockton,1504,5,N
5,252,Karl Malone,1476,6,N
6,708,Kevin Garnett,1462,7,N
7,788,Kevin Willis,1424,8,N
8,2544,LeBron James,1421,9,Y
9,1891,Jason Terry,1410,10,N


Thought: Include whether or not this player is top10 in a stat? Probably not. This is neat but likely not useful. 

### assistleaders

In [21]:
from nba_api.stats.endpoints import assistleaders

In [22]:
al = assistleaders.AssistLeaders(league_id='00',player_or_team='Player').get_data_frames()
pd.DataFrame(al[0])

,RANK,PLAYER_ID,PLAYER,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,JERSEY_NUM,PLAYER_POSITION,AST
0,1,1629027,Trae Young,1610612737,ATL,Atlanta Hawks,11,G,741
1,2,203999,Nikola Jokic,1610612743,DEN,Denver Nuggets,15,C,678
2,3,201935,James Harden,1610612755,PHI,Philadelphia 76ers,1,G,618
3,5,1627734,Domantas Sabonis,1610612758,SAC,Sacramento Kings,10,F-C,573
4,6,201566,Russell Westbrook,1610612746,LAC,LA Clippers,0,G,551


In [23]:
al = assistleaders.AssistLeaders(league_id='00',player_or_team='Team',season='2022-23').get_data_frames()
pd.DataFrame(al[0])

,RANK,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,AST
0,1,1610612744,GSW,Golden State Warriors,2446
1,2,1610612743,DEN,Denver Nuggets,2368
2,3,1610612758,SAC,Sacramento Kings,2237
3,4,1610612756,PHX,Phoenix Suns,2235
4,5,1610612759,SAS,San Antonio Spurs,2232


In [24]:
al = assistleaders.AssistLeaders(league_id='00',player_or_team='Team',season='2021-22').get_data_frames()
al

[   RANK     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME   AST
 0     1  1610612766               CHA      Charlotte Hornets  2302
 1     2  1610612759               SAS      San Antonio Spurs  2289
 2     3  1610612743               DEN         Denver Nuggets  2279
 3     4  1610612756               PHX           Phoenix Suns  2244
 4     5  1610612744               GSW  Golden State Warriors  2223]

Thoughts: Not useful. Assist leaders would be implied in aggregations. 

### assisttracker  

In [25]:
from nba_api.stats.endpoints import assisttracker

In [26]:
assisttracker.AssistTracker(team_id_nullable='1610612744',league_id_nullable='00',season_nullable='2022-23').get_data_frames()

[   ASSISTS
 0     2812]

Thought: I truly have no idea what information this is giving me. 

### boxscoreadvancedv2

In [14]:
from nba_api.stats.endpoints import boxscoreadvancedv2

In [16]:
bsa = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=gid1).get_data_frames()

In [27]:
bsa[0].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36.000000:24,...,10.5,0.750,0.750,0.186,0.192,101.33,100.22,83.52,76,0.149
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31.000000:22,...,14.3,0.364,0.364,0.171,0.177,101.27,101.76,84.80,67,0.041
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29.000000:38,...,0.0,0.650,0.650,0.143,0.145,102.79,102.05,85.04,62,0.095
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30.000000:24,...,16.7,0.778,0.810,0.164,0.168,102.38,101.84,84.87,64,0.110
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34.000000:33,...,14.3,0.375,0.389,0.345,0.356,102.79,101.40,84.50,74,0.066


In [18]:
bsa[1]

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0021800283,1610612742,Mavericks,DAL,Dallas,240.000000:00,113.9,115.3,103.3,106.1,...,16.3,0.575,0.587,1.0,0.198,99.98,98.0,81.67,98,0.553
1,0021800283,1610612738,Celtics,BOS,Boston,240.000000:00,103.3,106.1,113.9,115.3,...,18.4,0.535,0.567,1.0,0.197,99.98,98.0,81.67,98,0.447


Thought: Okay, this is very useful. This will probably form the basis of what I want to do.  

### boxscoredefensive  

In [28]:
#from nba_api.stats.endpoints import boxscoredefensive

In [29]:
#bsd = boxscoredefensive.BoxScoreDefensive(game_id=gid1).get_data_frames()

In [30]:
#bsd[0]

Thoughts: Didn't work when I tried it. 

### boxscorefourfactorsv2

In [31]:
from nba_api.stats.endpoints import boxscorefourfactorsv2

In [33]:
bsff = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id=gid1).get_data_frames()

In [36]:
bsff[0].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36.000000:24,0.566,0.074,0.168,0.214,0.493,0.246,0.170,0.317
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31.000000:22,0.567,0.100,0.163,0.222,0.526,0.224,0.155,0.273
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29.000000:38,0.564,0.073,0.185,0.182,0.644,0.269,0.161,0.231
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30.000000:24,0.554,0.143,0.180,0.208,0.657,0.259,0.142,0.231
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34.000000:33,0.524,0.159,0.186,0.222,0.631,0.246,0.165,0.241


In [56]:
# comparing with the boxscoreadvancedv2 endpoint 
bsa_comp = bsa[0]
comp_cols = []
for i in list(bsa_comp.columns):
    if i in list(bsff[0].columns):
        comp_cols.append(i)

In [54]:
bsa_comp[comp_cols].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,OREB_PCT,TM_TOV_PCT,EFG_PCT
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36.000000:24,0.000,10.5,0.750
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31.000000:22,0.032,14.3,0.364
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29.000000:38,0.038,0.0,0.650
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30.000000:24,0.000,16.7,0.778
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34.000000:33,0.061,14.3,0.375


In [55]:
bsff[0][comp_cols].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,OREB_PCT,TM_TOV_PCT,EFG_PCT
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36.000000:24,0.214,0.168,0.566
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31.000000:22,0.222,0.163,0.567
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29.000000:38,0.182,0.185,0.564
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30.000000:24,0.208,0.180,0.554
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34.000000:33,0.222,0.186,0.524


Thought: There is obviously something going on here to make them different, but I really have no clue how they arrived at this stuff. I don't know why they are different. I don't know what is going on. 

### boxscorematchups

In [63]:
#from nba_api.stats.endpoints import boxscorematchups

In [64]:
#bsmu = boxscorematchups.BoxScoreMatchups(gid1).get_data_frames()

In [65]:
# didn't work when I tried it 

### boxscoremiscv2

In [66]:
from nba_api.stats.endpoints import boxscoremiscv2

In [67]:
bsm = boxscoremiscv2.BoxScoreMiscV2(game_id=gid1).get_data_frames()

In [69]:
bsm[0].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36.000000:24,...,7.0,6.0,14.0,19.0,9.0,36.0,1.0,0.0,3.0,0.0
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31.000000:22,...,0.0,0.0,9.0,13.0,2.0,26.0,0.0,0.0,2.0,1.0
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29.000000:38,...,0.0,8.0,16.0,10.0,4.0,30.0,0.0,1.0,4.0,1.0
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30.000000:24,...,6.0,12.0,15.0,10.0,0.0,24.0,1.0,0.0,3.0,1.0
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34.000000:33,...,2.0,10.0,17.0,10.0,4.0,30.0,1.0,2.0,2.0,3.0


Thoughts: Really great stuff here, too. Using this in conjunction with the advanced stuff will be awesome. 

### boxscoreplayertrackv2

In [70]:
from nba_api.stats.endpoints import boxscoreplayertrackv2

In [71]:
bspt = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=gid1).get_data_frames()

In [74]:
bspt[0].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,SPD,...,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,F,,36:24,4.06,...,1,3,0.333,8,11,0.727,0.643,0,2,0.000
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,F,,31:22,3.71,...,0,0,0.000,3,11,0.273,0.273,4,6,0.667
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,C,,29:38,4.17,...,5,6,0.833,1,4,0.250,0.599,3,3,1.000
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,G,,30:24,4.18,...,3,3,1.000,4,6,0.667,0.778,2,3,0.667
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,G,,34:33,3.97,...,5,11,0.455,4,13,0.308,0.375,1,1,1.000


Thoughts: I don't know what most of this means, but I'm gonna use it. 

## Some Overall Thoughts

As it sits right now, I have about 75 more modules to check. I'm not going to do that. There is simply an overwhelming amount of data available. I'm going to click through them and note the modules below that I think can be useful. 